## ETL pipeline


In [21]:
# import necessary packages

import requests # help to pull data from an API
import pandas as pd # treatment of data
from sqlalchemy  import create_engine # connection with database

## (E)xtract

Extraction of data from some source

In [22]:
def extract()-> dict:
    API_URL = "http://universities.hipolabs.com/search?country=Brazil"
    data = requests.get(API_URL).json()
    return data


## (T)ransform

Transformation of data acquired on extract phase.

In [23]:
def transform(data:dict) -> pd.DataFrame:
    df = pd.DataFrame(data)
    print(f"Total Number of universities from API {len(data)}")
    df = df[df["name"].str.contains("Universidade Federal")]
    print(f"Number of federal universities in Brazil {len(df)}")
    df['domains'] = [','.join(map(str, l)) for l in df['domains']]
    df['web_pages'] = [','.join(map(str, l)) for l in df['web_pages']]
    df = df.reset_index(drop=True)
    return df[["domains","country","web_pages","name"]]


## (L)oad

Last step of pipeline. Load the data after the treatment.

In [24]:
def load(df:pd.DataFrame)-> None:
    """ Loads data into a database"""
    disk_engine = create_engine('sqlite:///my_lite_store.db')
    df.to_sql('cal_uni', disk_engine, if_exists='replace')


## ETL

Run all functions.

In [25]:
data = extract()
df = transform(data)
load(df)


Total Number of universities from API 175
Number of federal universities in Brazil 44
